# Gate Set Tomography

*Copyright (c) 2022 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline

Recall Quantum State Tomography (QST) and Quantum Process Tomography (QPT), they assume the initial states and final measurements are known. These states and measurements must be prepared using quantum gates which themselves may be faulty. Thus, QST and QPT have a self-consistency problem. To completely characterize an unknown set of gates and states, we can use Gate Set Tomography (GST).

This tutorial introduces Gate Set Tomography (GST), covering its theory and implementation on [Baidu Quantum Platform](https://quantum.baidu.com/).

## Theory



 ### Gate set

Firstly, we define a gate set as

$$
    \mathcal{G} := \left\{ |\rho\rangle\rangle, \langle\langle E|, \mathsf{G}_0, \cdots, \mathsf{G}_K\right\},
$$

where $|\rho\rangle\rangle$ is the PTM of initial state $\rho$, $\langle\langle E|$ is the PTM of a $2$-outcome POVM, and each $\mathsf{G}_k$ is the PTM of gate $G_k$. This means we pack all the information we are interested in. Sometimes we would like to focus only on the gates and not the states and measurements, and we write the gate set as

$$
    \mathcal{G} := \left\{\mathsf{G}_0, \cdots, \mathsf{G}_K\right\}.
$$

The distinction should be clear from the context.

As in QPT, the information needed to reconstruct each gate $G_k$ is contained in measurements of $\langle\langle E_i|\mathsf{G}_k|\rho_j\rangle\rangle$, the gate of interest sandwiched between a complete set of states and POVMs. The experimental requirements for GST are similar to QPT, but for self-consistency, we must treat these state preparation and measurement (SPAM) gates on the same footing as the original gates $\left\{\mathsf{G}_k\right\}$.

Since, we define SPAM gates as

$$
    \mathcal{F} := \left\{ \mathsf{F}_1, \cdots, \mathsf{F}_N \right\},
$$

where $\mathsf{F}_i$ is composed of gates in gates $\left\{\mathsf{G}_k\right\}$, i.e.,

$$
    \mathsf{F}_i = \mathsf{G}_{i_1} \circ \mathsf{G}_{i_2} \circ \cdots \circ \mathsf{G}_{i_{L_i}},
$$

where $L_i$ is the length of the $i$-th SPAM gate.


### Linear inversion method

Then, we will introduce a simple algorithm for obtaining self-consistent gate estimates, named Linear-inversion GST.

For an $n$-qubit system, we begin by identifying a set of SPAM gate strings $\mathcal{F} = \left\{\mathsf{F}_1, \cdots, \mathsf{F}_{4^n}\right\}$ that, when applied to our unknown fixed state $|\rho\rangle\rangle$ and measurement $\langle\langle E|$, produce a complete set of initial state $|\rho_j\rangle\rangle=\mathsf{F}_j|\rho\rangle\rangle$, and final states $\langle\langle E_i| = \langle\langle E|\mathsf{F}_i$.

#### Example gate set

It is useful to have concrete examples of gate sets. Here we consider the simplest example, and we will implement it on [Baidu Quantum Platform](https://quantum.baidu.com/) latter.

* $\mathcal{G} = \left\{\left\{\right\}, X_{\pi/2}, Y_{\pi/2}\right\} = \left\{\mathsf{G}_0, \mathsf{G}_1, \mathsf{G}_2\right\}$
* $\mathcal{F} = \left\{\left\{\right\}, X_{\pi/2}, Y_{\pi/2}, X_{\pi/2} \circ X_{\pi/2}\right\} = \left\{\left\{\right\}, \mathsf{G}_1, \mathsf{G}_2, \mathsf{G}_1 \circ \mathsf{G}_1\right\}$

where $X_{\pi/2}$ is a $\pi/2$ rotation about the $X$-axis of the Bloch sphere, and the symbol $\{\}$ denotes the "null" gate - do nothing for no time. (We will always choose $G_0$ to be the "null" gate for reason will be clear later.) It is easy to see that the set of states $\mathsf{F}_j|\rho\rangle\rangle$ (measurements $\langle\langle E|\mathsf{F}_i$) spans the Bloch sphere.

#### Data collection

In GST, we work with expectation values,

$$
    p_{ikj} = \langle\langle E|\mathsf{F}_i\mathsf{G}_k\mathsf{F}_j|\rho\rangle\rangle,
$$

where $\mathsf{F}_i,\mathsf{F}_j\in\mathcal{F}$ and $\mathsf{G}_k\in\mathcal{G}$. The corresponding circuit is shown as follows.

![GST](./figures/gst-general-circuit.png "Figure 1: We construct such circuits to estimate expectation values. ")

Inserting the complete set of Pauli basis, we have

$$
\begin{align}
    p_{ikj}
&=  \sum_{rs} \langle\langle E | \mathsf{F}_i | r \rangle\rangle\langle\langle r | \mathsf{G}_k | s \rangle\rangle\langle\langle s |\mathsf{F}_j | \rho \rangle\rangle \\
&=  \sum_{rs} A_{ir} \left( \mathsf{G}_k\right)_{rs} B_{sj}.
\end{align}
$$

We can write the above equation in the form of the matrix as follows,

$$
    \widetilde{\mathsf{G}}_k = A \mathsf{G}_k B,
$$

where $A = \sum_i | i \rangle\rangle\langle\langle E | \mathsf{F}_i $, $B = \sum_j \mathsf{F}_j | \rho \rangle\rangle\langle\langle j |$, and $\left(\widetilde{\mathsf{G}}_k\right)_{ij} = p_{ikj}$. Since the question becomes how to estimate $\mathsf{G}_k$ with experimental data $\widetilde{\mathsf{G}}_k$.

Note that we have defined $\mathsf{G}_0 = \left\{\right\}$ before, then we define a matrix $g$, named *Gram matrix*,

$$
  g := \widetilde{\mathsf{G}}_0 = A B.
$$

We observe that

$$
  g^{-1}\widetilde{\mathsf{G}}_k = B^{-1} A^{-1} A \mathsf{G}_k B = B^{-1} \mathsf{G}_k B.
$$

Then we have,

$$
  \hat{\mathsf{G}}_k = g^{-1} \widetilde{\mathsf{G}}_k,
$$

is an estimation of the gate set, up to similarity transformation by the unobservable matrix $B$. And the question becomes how to get this unobservable matrix $B$.


#### Gauge optimization

Now consider the vectors

$$
\begin{align}
    | \widetilde{\rho} \rangle\rangle := A | \rho \rangle\rangle
&=  \sum_i | i\rangle\rangle \langle\langle E | \mathsf{F}_i | \rho \rangle\rangle, \\
    \langle\langle \widetilde{E} | := \langle\langle E | B
&=  \sum_j \langle\langle E | \mathsf{F}_j | \rho \rangle\rangle \langle\langle j |.
\end{align}
$$

Note that both $| \widetilde{\rho} \rangle\rangle$ and $\langle\langle \widetilde{E} |$ are measurable, and $\langle\langle i|\widetilde{\rho}\rangle\rangle = \langle\langle\widetilde{E}|i\rangle\rangle = \langle\langle E|\mathsf{F}_i|\rho\rangle\rangle$, the corresponding circuits are as follows.

![GST](./figures/gst-circuit-new-gateset.png "Figure 2: We construct such circuits to get new gate set.")

And then, we construct a new gate set,

$$
\begin{align}
  &\hat{\mathcal{G}} := \left\{|\hat{\rho}\rangle\rangle, \langle\langle\hat{E}|, \left\{ \hat{G}_k\right\}\right\} =
  \left\{B^{-1}| \rho \rangle\rangle, \langle\langle E | B, \left\{ B^{-1} G_k B\right\}\right\}, \\
  &| \hat{\rho} \rangle\rangle := g^{-1} | \widetilde{\rho} \rangle\rangle = B^{-1} |\rho\rangle\rangle, \;
  \langle\langle \hat{E} | := \langle\langle \widetilde{E} | = \langle\langle E | B, \;
  \hat{G}_k = B^{-1} G_k B.
\end{align}
$$

For this new gate set, we can estimate $|\hat{\rho}\rangle\rangle$ and $\langle\langle\hat{E}|$ with above circuit.

Then, we will show that how to estimate matrix $B$ with this new gate set. **Based on high quality quantum device**, we know a priori that the measured gates will differ from an ideal (target) set of gates by some very small error. Since given a target set of gates $\mathcal{G}^t = \left\{ | \rho^t \rangle\rangle, \langle\langle E^t |, \left\{ G_k^t\right\} \right\}$ and our *Linear inversion GST* estimate $\hat{\mathcal{G}} = \left\{|\hat{\rho}\rangle\rangle, \langle\langle\hat{E}|, \left\{ \hat{G}_k\right\}\right\}$, we find the matrix $B^e$, which is our estimation of $B$,

$$
  B^e = \textrm{argmin}_{B} \sum_{k=0}^{K+1} \textrm{Tr} \left\{ \left( \hat{\mathsf{G}}_k - B^{-1} \mathsf{G}_k^t B \right)^T \left( \hat{\mathsf{G}}_k - B^{-1} \mathsf{G}_k^t B \right)\right\},
$$

where we define $\mathsf{G}_{K+1} = | \rho \rangle\rangle \langle\langle E |$. Since we can get the estimation of original gate set with following equations,

$$
\begin{align}
  &| \rho^e \rangle\rangle = B^e|\hat{\rho}\rangle\rangle, \\
  &\langle\langle E^e | = \langle\langle \hat{E} | \left(B^e\right)^{-1}, \\
  &\mathsf{G}^e_k = B^e \hat{\mathsf{G}}_k \left(B^e\right)^{-1}.
\end{align}
$$

## Practice

We demonstrate GST on the single qubit case.

First, we import the necessary libraries.

In [5]:
import QCompute
import numpy as np
from qcompute_qep.tomography import GateSetTomography, GateSet
import qcompute_qep.tomography as tomography
from QCompute.QPlatform.QOperation import CircuitLine

Then we set up the Gate Set we are interested in.

* $\mathcal{G} = \left\{\left\{\right\}, X_{\pi/2}, Y_{\pi/2}\right\} = \left\{\mathsf{G}_0, \mathsf{G}_1, \mathsf{G}_2\right\}$
* $\mathcal{F} = \left\{\left\{\right\}, X_{\pi/2}, Y_{\pi/2}, X_{\pi/2} \circ X_{\pi/2}\right\} = \left\{\left\{\right\}, \mathsf{G}_1, \mathsf{G}_2, \mathsf{G}_1 \circ \mathsf{G}_1\right\}$

In [6]:
prep_circuit = []
meas_circuit = []

rx_90 = CircuitLine(QCompute.RX(np.pi / 2), [0])
ry_90 = CircuitLine(QCompute.RY(np.pi / 2), [0])


gate_set = GateSet(prep_circuit, meas_circuit, gates={'rx_90': rx_90, 'ry_90': ry_90},
                   prep_gates=[['rx_90'], ['ry_90'], ['rx_90', 'rx_90']],
                   meas_gates=[['rx_90'], ['ry_90'], ['rx_90', 'rx_90']])

Then we set the quantum computer (instance of QComputer). The QuantumComputer can be a simulator or a hardware interface. The rest is simple, we initialize a GateSetTomography instance, call the tomography procedure and obtain the noisy gate set. Here, we set `qubits=[1]` that means we are interested in second qubit of quantum computer.

In [7]:
# Please login the "Quantum Leaf" platform (https://quantum-hub.baidu.com/) to get Token
QCompute.Define.hubToken = "Token"

# qc = QCompute.BackendName.CloudBaiduQPUQian
qc = QCompute.BackendName.LocalBaiduSim2

tomo = GateSetTomography()

gate_set = tomo.fit(qc, gate_set=gate_set, qubits=[1])

Successfully finished GST!: 100%|███████████| 100.0/100 [00:02<00:00, 45.66it/s]


Finally, we can analyze the experimental data and visualize these PTMs to see the effect of GST.

In [8]:

for i, key in enumerate(gate_set.noisy_gate.keys()):
    print(key)
    # print(gate_set.noisy_gate[key])
    ideal = tomo.result['ideal gateset'][i]
    noisy = gate_set.noisy_gate[key]
    tomography.compare_process_ptm(ptms=[ideal, noisy, ideal - noisy])
    print("The average gate fidelity between these two PTMs is: {}".format(gate_set.fidelity(key)))
    print("**************************")

print("The state fidelity is: {}".format(gate_set.fidelity('rho')))
print("***************")
print("The measurement fidelityis: {}".format(gate_set.fidelity('meas')))

rx_90
The average gate fidelity between these two PTMs are: 1.0032814481981147
**************************
ry_90
The average gate fidelity between these two PTMs are: 1.003774062117967
**************************
The state fidelity is: 1.0075878686910356
***************
The measurement fidelityis: 0.9923411588448733


## References
[1] Greenbaum, Daniel. "Introduction to quantum gate set tomography." [arXiv](https://arxiv.org/abs/1509.02921) preprint arXiv:1509.02921 (2015).